In [1]:
# Description - Compression Comparison Between Models:{Factorized prior balle_variational_2018, Hyperprior balle_variational_2018,
# Hyperprior with a Gaussian mixture model minnen_joint_2018,Joint autoregressive and hyperprior minnen_joint_2018
# Extension from minnen_joint_2018, residual blocks and sub-pixel deconvolution cheng_learned_2020}

import math
import io
import torch
from torchvision import transforms
import numpy as np

from PIL import Image

import matplotlib.pyplot as plt

In [2]:
from pytorch_msssim import ms_ssim

In [3]:
from compressai.zoo import (bmshj2018_factorized, bmshj2018_hyperprior, mbt2018_mean, mbt2018, cheng2020_anchor)

In [4]:
from ipywidgets import interact, widgets

In [7]:
def compute_psnr(a, b):
    mse = torch.mean((a - b)**2).item()
    return -10 * math.log10(mse)

def compute_msssim(a, b):
    return ms_ssim(a, b, data_range=1.).item()

def compute_bpp(out_net):
    size = out_net['x_hat'].size()
    num_pixels = size[0] * size[2] * size[3]

    return sum(torch.log(likelihoods).sum() / (-math.log(2) * num_pixels)
            for likelihoods in out_net['likelihoods'].values()).item()

In [8]:
import sys
import os
import time
from statistics import mean
def get_reconstructions(networks, directory, quality):
    reconstructed_images = []
    metric_list = []
    bpp_list = []
    for images in os.listdir(directory):
        img = Image.open(directory + "/" + images).convert("RGB").resize((1088,1920))
        x = transforms.ToTensor()(img).unsqueeze(0).to(device)
        outputs = {}
        with torch.no_grad():
            for name, net in networks.items():
                rv = net(x)
                rv['x_hat'].clamp_(0, 1)
                rv['x_hat']
                outputs[name] = rv
        d_npy_xhat = rv['x_hat'].cpu().numpy()

        reconstructions = {name: transforms.ToPILImage()(out['x_hat'].squeeze().cpu())
                           for name, out in outputs.items()}
        metrics = {}
        name_metric = {}
        metrices_dict = []
        
        for name, out in outputs.items():
            metric_list.append(compute_psnr(x, out["x_hat"]))
            bpp_list.append(compute_bpp(out))
    print(mean(metric_list), mean(bpp_list))

In [10]:
#Load Pre-Trained Networks
quality = 1
for i in range(6):
    r_images = []
#     directory = "/trinity/home/a.jha/scripts/Image_Compression/P-Frames/H26520/"
    directory = "/trinity/home/a.jha/scripts/Image_Compression/kodak_images/"
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    metric = 'mse'  # only pre-trained model for mse are available for now
    
    networks = {
#         'cheng2020-anchor': cheng2020_anchor(quality=quality, pretrained=True).eval().to(device),
#         'bmshj2018-factorized': bmshj2018_factorized(quality=quality, pretrained=True).eval().to(device),
        'bmshj2018-hyperprior': bmshj2018_hyperprior(quality=quality, pretrained=True).eval().to(device),
#         'mbt2018-mean': mbt2018_mean(quality=quality, pretrained=True).eval().to(device),
#         'mbt2018': mbt2018(quality=quality, pretrained=True).eval().to(device),
    }
    get_reconstructions(networks, directory, quality)
    quality = quality + 1

31.48431547574274 0.08393712403873603
33.29889857999003 0.11796063681443532
35.187170397738726 0.16024538036435843
37.03642704702601 0.2175674857571721
38.778880416717236 0.2906538049379985
40.58618257037663 0.3829578850418329
